In [23]:
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
import folium 

In [25]:
df_4 = pd.read_csv('feux_de_foret.csv',usecols=['FIRE_SIZE','STATE','STAT_CAUSE_DESCR'])
df_group1 = df_4.groupby('STATE',as_index=False).agg({'FIRE_SIZE':'sum'})
df_group2 = df_4.groupby('STATE',as_index=False).agg({'FIRE_SIZE':'count'})
df_group = pd.merge(left=df_group1,right=df_group2,how='left',left_on='STATE',right_on='STATE')
df_group = df_group.rename(columns={'STATE':'Identifiant','FIRE_SIZE_x':"Etendue totale des feux de forêt",'FIRE_SIZE_y':"Nombre total de feux de forêt"})

In [32]:
df_group.head()

,Identifiant,Etendue totale des feux de forêt,Nombre total de feux de forêt
0,AK,3.223309e+07,12843
1,AL,9.205453e+05,66570
2,AR,5.089118e+05,31663
3,AZ,5.576681e+06,71586
4,CA,1.274586e+07,189550


In [26]:
state_geo = 'us-states.json'
state_data = pd.read_csv('feux_de_foret.csv')

C:\Users\pierr\AppData\Local\Temp\ipykernel_8652\1485460832.py:2: DtypeWarning:

Columns (9,11,12,13,14,15,16,17,18,19,36,38) have mixed types. Specify dtype option on import or set low_memory=False.



In [27]:
#Creation of a map of the States most affected by forest fires

m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=df_group,
    columns=["Identifiant", "Nombre total de feux de forêt"],
    key_on="feature.id",
    fill_color="OrRd",
    fill_opacity=0.5,
    line_opacity=0.2,
    legend_name="Nombre de feux de forêt entre 1992 et 2015",
).add_to(m)


#df.apply(lambda row:folium.Marker(location=[row["LATITUDE"], row["LONGITUDE"]]).add_to(m),axis=1)



folium.LayerControl().add_to(m)






m

In [28]:
dfeux = pd.read_csv('dfeux.csv')
dfeux = dfeux.groupby(['ANNEE','Etat','Identifiant'],as_index=False)['Température'].mean()
df_2 = pd.read_csv('df_IDState_Etendue_Nombre_feux_par_an.csv')
df_or = pd.read_csv('feux_de_foret.csv',usecols=['FIRE_SIZE','FIRE_YEAR','STATE','STAT_CAUSE_DESCR'])
df_ori = df_or.groupby(['STATE','FIRE_YEAR'],as_index=False).agg({'FIRE_SIZE':'sum','STAT_CAUSE_DESCR':'count'})
df_orin = df_ori.groupby(['STATE','FIRE_YEAR','FIRE_SIZE'],as_index=False).agg({'STAT_CAUSE_DESCR':'sum'})



In [29]:
df_orin = df_orin.sort_values(['FIRE_YEAR','STATE'])
dfeux = dfeux.sort_values(['ANNEE','Etat'])
df_final = pd.merge(df_orin,dfeux,how='inner',left_on=['STATE','FIRE_YEAR'],right_on=['Identifiant','ANNEE'])
df_final = df_final.drop(['FIRE_YEAR','STATE'],axis=1).rename(columns={'FIRE_SIZE':'Etendue brûlée','STAT_CAUSE_DESCR':'Nombre de feux','ANNEE':'Année'})

In [30]:
etats_co = pd.read_csv('Etats_coor.csv')
etats_co['Id'] = ['AL',
'AK',
'AZ',
'AR',
'CA',
'CO',
'CT',
'DE',
'DC',
'FL',
'GA',
'HI',
'ID',
'IL',
'IN',
'IA',
'KS',
'KY',
'LA',
'ME',
'MD',
'MA',
'MI',
'MN',
'MS',
'MO',
'MT',
'NE',
'NV',
'NH',
'NJ',
'NM',
'NY',
'NC',
'ND',
'OH',
'OK',
'OR',
'PA',
'RI',
'SC',
'SD',
'TN',
'TX',
'UT',
'VT',
'VA',
'WA',
'WV',
'WI',
'WY']
etats_coor = pd.merge(left=etats_co,right=df_group,how='left',left_on='Id',right_on='Identifiant')


In [31]:
#Addition of markers corresponding to the extent of forest fires in each State
for i in range(0,len(etats_coor)):
   folium.CircleMarker(
      location=[etats_coor.iloc[i]['LAT'], etats_coor.iloc[i]['LONG']],
      popup=etats_coor.iloc[i]['ETAT'],
      radius=etats_coor.iloc[i]['Etendue totale des feux de forêt']/700000,
      color='crimson',
      fill=True,
      fill_color='crimson',
      
   ).add_to(m)

m